# Quadratic Programming: Penalty Method

In [7]:
import jax
import jax.numpy as jnp
from chex import Array
from jax import grad, jit

# Energy cost matrix (actuator 1 is less efficient)
P = jnp.array([[3.0, 0.5], [0.5, 2.0]])
q = jnp.array([0.5, -0.3])
d_ref = 1.0  # Target distance to cover


def qp_objective(x: Array) -> Array:
    """QP objective: 0.5 * x^T P x + q^T x"""
    return 0.5 * jnp.dot(x, jnp.dot(P, x)) + jnp.dot(q, x)


def qp_penalized(x: Array, rho: float) -> Array:
    """Penalized QP objective"""
    obj = qp_objective(x)
    # Equality constraint: x1 + x2 = d_ref
    eq_penalty = (x[0] + x[1] - d_ref) ** 2
    # Inequality constraints: x1 >= 0, x2 >= 0 (rewritten as -x <= 0)
    ineq_penalty = jnp.sum(jnp.maximum(0, -x) ** 2)
    return obj + rho * (eq_penalty + ineq_penalty)


grad_qp = jit(grad(qp_penalized))

learning_rate = 0.001
max_iters = 5000
tol = 1e-8
rho = 50.0

x = jnp.array([0.5, 0.5])  # Initial guess

for i in range(max_iters):
    g = grad_qp(x, rho)
    x_new = x - learning_rate * g
    if jnp.linalg.norm(x_new - x) < tol:
        break
    x = x_new

print(f"Converged after {i} iterations.")
print(f"Optimal effort allocation: {x}")
print(f"Total effort (should be {d_ref}): {x[0] + x[1]:.4f}")
print(f"Energy cost: {qp_objective(x):.4f}")
# Converged after 4999 iterations.
# Optimal effort allocation: [0.16969994 0.8161287 ]
# Total effort (should be 1.0): 0.9858
# Energy cost: 0.6185

Converged after 4999 iterations.
Optimal effort allocation: [0.16969994 0.8161287 ]
Total effort (should be 1.0): 0.9858
Energy cost: 0.6185


# Quadratic Programming: JAXOpt

In [8]:
import jax.numpy as jnp
from jaxopt import OSQP

# Define the QP parameters
P = jnp.array([[3.0, 0.5], [0.5, 2.0]])
q = jnp.array([0.5, -0.3])
d_ref = 1.0

# Equality constraint: x1 + x2 = d_ref
A_eq = jnp.array([[1.0, 1.0]])
b_eq = jnp.array([d_ref])

# Inequality constraints: -x1 <= 0, -x2 <= 0 (i.e., x >= 0)
G = jnp.array([[-1.0, 0.0], [0.0, -1.0]])
h = jnp.array([0.0, 0.0])

# Solve with OSQP
solver = OSQP()
result = solver.run(params_obj=(P, q), params_eq=(A_eq, b_eq), params_ineq=(G, h))

print(f"Optimal effort allocation: {result.params.primal}")
print(
    f"Energy cost: {0.5 * result.params.primal @ P @ result.params.primal + q @ result.params.primal:.4f}"
)

Optimal effort allocation: [0.17520773 0.82478786]
Energy cost: 0.6387
